In [1]:
import pandas as pd
import os

In [2]:
cd = "C:\\Users\\ASUS\\Desktop\\data"

In [3]:
os.listdir(cd)

['.Rhistory',
 'A1_.csv',
 'A1_FE.xlsx',
 'A1_imp.csv',
 'A1_RF.csv',
 'A2_.csv',
 'A2_FE.xlsx',
 'A2_imp.csv',
 'A2_RF.csv',
 'correlation.csv',
 'CVndepthA1.png',
 'CVndepthA2.png',
 'CVndepthQ1.png',
 'CVndepthQ2.png',
 'CVndepthQ3.png',
 'CVnpercA1.png',
 'CVnpercA2.png',
 'CVnpercQ1.png',
 'CVnpercQ2.png',
 'CVnpercQ3.png',
 'CVnpredA1.png',
 'CVnpredA2.png',
 'CVnpredQ1.png',
 'CVnpredQ2.png',
 'CVnpredQ3.png',
 'CVNTreesA1.png',
 'CVNTreesA2.png',
 'CVNTreesQ1.png',
 'CVNTreesQ2.png',
 'CVNTreesQ3.png',
 'FeatureImportanceA1.png',
 'FeatureImportanceA2.png',
 'FeatureImportanceQ1.png',
 'FeatureImportanceQ2.png',
 'FeatureImportanceQ3.png',
 'FF.csv',
 'Figure_4_6_Table1.R',
 'FMret_avg1.xlsx',
 'FMret_score1.xlsx',
 'forecast.R',
 'ipi.csv',
 'PDP_Linear_Error_MEANEST.png',
 'PDP_Linear_Error_past_eps.png',
 'Q1_.csv',
 'Q1_FE.xlsx',
 'Q1_imp.csv',
 'Q1_RF.csv',
 'Q2_.csv',
 'Q2_FE.xlsx',
 'Q2_imp.csv',
 'Q2_RF.csv',
 'Q3_.csv',
 'Q3_FE.xlsx',
 'Q3_imp.csv',
 'Q3_RF.csv',
 'ret

In [4]:
os.chdir(cd)

In [5]:
CRSP = pd.read_csv("C:\\Users\\ASUS\\Desktop\\bias\\all_datasets\\crsp_msf.csv",usecols=['PERMNO', 'date', 'SHRCD', 'EXCHCD'], low_memory=False)

In [6]:
CRSP = CRSP[(CRSP.SHRCD.apply(lambda x: x in [10.0, 11.0])) & (CRSP.EXCHCD.apply(lambda x: x in [1, 2, 3]))].reset_index(drop=True)

In [7]:
CRSP['rankdate'] = pd.to_datetime(CRSP.date,format='%Y%m%d')
CRSP['rankdate'] = CRSP['rankdate'].dt.to_period('M')

In [8]:
CRSP.drop(['date', 'SHRCD', 'EXCHCD'], axis=1, inplace=True)

In [9]:
#####import data of RF forecast######
A1_RF_forecast= pd.read_csv('A1_RF.csv', low_memory=False)
A2_RF_forecast= pd.read_csv('A2_RF.csv', low_memory=False)
Q1_RF_forecast= pd.read_csv('Q1_RF.csv', low_memory=False)
Q2_RF_forecast= pd.read_csv('Q2_RF.csv', low_memory=False)
Q3_RF_forecast= pd.read_csv('Q3_RF.csv', low_memory=False)

In [10]:
start_date='1986-01'
end_date='2021-12'
feature_names=['predict_rf_mean','analyst_forecast','adj_actual','RF_dif','af_dif','sqr_RF_dif','sqr_af_dif','bias']
#####meger forecast bias with CRSP######
for horizon in ('Q1_','Q2_','Q3_','A1_','A2_'):
    Forecast_Output = eval(horizon + 'RF_forecast').reset_index(drop=True)
    Forecast_Output.rename(columns={'MEANEST': 'analyst_forecast'}, inplace=True)
    output = horizon + 'FE.xlsx'
    Forecast_Output = Forecast_Output[(Forecast_Output['rankdate'] >= start_date) & (Forecast_Output['rankdate'] <= end_date)]
    Forecast_Output['rankdate'] = pd.to_datetime(Forecast_Output.rankdate).dt.to_period('M')
    Forecast_Output['RF_dif'] = Forecast_Output['predict_rf_mean'] - Forecast_Output['adj_actual']
    Forecast_Output['af_dif'] = Forecast_Output['analyst_forecast'] - Forecast_Output['adj_actual']
    Forecast_Output['sqr_af_dif'] = Forecast_Output['af_dif'] ** 2
    Forecast_Output['sqr_RF_dif'] = Forecast_Output['RF_dif'] ** 2
    Forecast_Output[horizon+'dif'] = Forecast_Output['analyst_forecast'] - Forecast_Output['predict_rf_mean']
    Forecast_Output = pd.merge(left=CRSP, right=Forecast_Output[['PERMNO','rankdate','PRC', 'predict_rf_mean','analyst_forecast','adj_actual',
                                                                 'RF_dif','af_dif','sqr_af_dif','sqr_RF_dif',horizon+'dif']], how='inner', on=['PERMNO','rankdate'])
    Forecast_Output['bias']= Forecast_Output[horizon + 'dif'] / Forecast_Output['PRC']
    FE = Forecast_Output.groupby(['rankdate'])[feature_names].mean()
    FE['count'] = Forecast_Output['predict_rf_mean'].count()
    FE.to_excel(output)  ### Output monthly aggregate forecast errors